# Trabajo Final procesamiento de lenguaje natural (NLP) <a class="tocSkip">
## Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** Fabian Urrego

**ID:** 000059399



**Subir Este notebook de Trabajo Final en su repositorio de GitHub. y enviar el link en **Microsoft TEAMS**

Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo
Realizar un proceso de Procesamiento de lenguaje natural (NLP) para dejar los datos preparados para ser usados con algoritmos de Machine Learning para Clasificación como objetivo final del trabajo.

los datos preparados para ser usados con algoritmos de Machine Learning para Regresión o Clasificación como objetivo final del trabajo.

El trabajo se realizara en este jupyter notebook y subirlo a su repositorio de github creado en clase. (**Recuerde poner su nombre e información**)

## Las actividades a realizar
    

1) Limpiar los datos de texto.

   - https://joserzapata.github.io/courses/nlp/procesamiento-basico/

   - https://joserzapata.github.io/courses/nlp/preprocesamiento-texto/

2) Realizar la representación de texto:

   - https://joserzapata.github.io/courses/nlp/representaciones/

   - Tokenización
   - Lematización o stemming
   - Representación de los datos de texto (Bag of Words o TF-IDF)

3) Utilice un modelo de Machine Learning para clasificación para entrenar y evaluar el modelo.
        
    - Calcule estas las métricas de evaluación (accuracy, precision, recall, f1-score) - https://joserzapata.github.io/courses/python-ciencia-datos/clasificacion/#evaluacion-modelo-simple


    
   
    
*NOTA: No dude en contactarme para cualquier pregunta o inquietud :) por el chat de Teams o al correo
joser.zapata@upb.edu.co*

## EVALUACIÓN


|Porcentaje en la evaluación | Descripción| Nada | Incompleto | Completo 
| :---: |:---: |:---: |:---: |:---: 
| 5 % |**limpieza de los datos** |
| 5 % | **Representacion de los datos** |
| 5 % | **Machine Learning** <br> Entrenar y evaluar el modelo propuesto |


In [1]:
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)
from sklearn.model_selection import train_test_split

Carga del dataset

El archivo  se carga en aproximadamente 10 segundos

In [3]:
# Usar su ID UPB Ejemplo: "0028984798"
id_upb = "000059399"

data_reviews = pd.read_parquet(
    "https://www.dropbox.com/scl/fi/gvk9yj8cn96oocr9z058x/filmaffinity_reviews_dataset.parquet?rlkey=xgvr00zvkxbkwqqavqutpsshg&st=xjb7xze9&dl=1"
)
data_reviews = data_reviews.sample(n=50_000, random_state=int(id_upb))
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 118683 to 96414
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author_review_desc  50000 non-null  object
 1   author_rating       50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


Ejemplo de algunas filas del dataset

In [4]:
data_reviews.sample(5)

,author_review_desc,author_rating
95713,\nPelícula de ciencia ficción en la que la Tie...,5
129952,\nUna antigua compañera de trabajo me dijo que...,2
35628,\n“Así continuaron viviendo en una realidad es...,10
118975,\nExitosa ficción emitida por Televisión Españ...,7
20527,"\nEl territorio de la fábula, de la parábola, ...",8


Evaluar los valores nulos

In [5]:
data_reviews.isnull().sum()

author_review_desc    0
author_rating         0
dtype: int64

En este caso no hay nulos.

Revisar si hay valores duplicados:

In [6]:
data_reviews.duplicated().sum()

np.int64(1)

No hay  valores duplicados.

## Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎



Definición: Se considera reseña positiva cuando la puntuación ("author_rating") es mayor que 6; negativa en caso contrario.

Crear la variable binaria de sentimiento: 1 (positivo) si author_rating > 6, 0 (negativo) de lo contrario

In [7]:
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

In [8]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]


Dividir el dataset en entrenamiento y prueba

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    stratify=y_data,  # Mantener la proporción de clases en ambos conjuntos
    random_state=42,
)

## Análisis exploratorio de los datos

para determinar que tipo de limpieza se debe realizar a los datos de texto de `X_train`

In [10]:
display(X_train)


81366     \nEn el festival Grec de Barcelona del año 201...
84717     \nVi primero la de 1953 y luego ésta. Mejora e...
132850    \nCual civilización condenada a la extinción, ...
60505     \nEn pleno siglo XXI no es de extrañar el topa...
124468    \n¿Para qué reunir a todos estos actores y hac...
                                ...                        
99934     \nLo que hace que la película funcione, y muy ...
53796     \nNuestra señora de Atocha. No cuelgue. Condes...
121255    \nTres años después de los sucesos de "Jaws", ...
118597    \nPelícula basada en el pasado romano, buena m...
139664    \nTodo el argumento gira en torno al personaje...
Name: author_review_desc, Length: 40000, dtype: object

## 1) Limpieza de los datos de texto

Tomar los datos de `X_train` y aplicar las funciones de limpieza que considere necesarias

Ayudas:

- Convertir a minúsculas
- Eliminar caracteres especiales y números
- Corregir palabras mal escritas
- etc


In [17]:
"""
SCRIPT PARA LIMPIAR X_TRAIN
============================

Aplica limpieza de texto a X_train:
✅ Convertir a minúsculas
✅ Eliminar caracteres especiales
✅ Eliminar números
✅ Corregir palabras mal escritas (abreviaturas comunes)
✅ Eliminar URLs, emails, menciones
✅ Eliminar espacios múltiples

USO DIRECTO CON TU X_TRAIN:
----------------------------
>>> from limpiar_X_train import limpiar_X_train
>>> X_train_limpio = limpiar_X_train(X_train)
"""

import re
import numpy as np


class LimpiadorTexto:
    """Limpia texto aplicando todas las transformaciones necesarias"""

    def __init__(self):
        # Diccionario de correcciones de palabras mal escritas
        self.correcciones = {
            # Abreviaturas comunes
            "q": "que",
            "xq": "porque",
            "pq": "porque",
            "tmb": "también",
            "tb": "también",
            "x": "por",
            "d": "de",
            "k": "que",
            "bn": "bien",
            "mxo": "mucho",
            "msj": "mensaje",
            "msg": "mensaje",
            # Risas
            "jajaja": "jaja",
            "jajajaja": "jaja",
            "jjjjj": "jaja",
            "jeje": "jaja",
            "jiji": "jaja",
            "ajaj": "jaja",
            "jejeje": "jaja",
            "jijiji": "jaja",
            # Otras comunes
            "xd": "",
            "lol": "",
        }

    def a_minusculas(self, texto):
        """Convierte texto a minúsculas"""
        return texto.lower()

    def eliminar_urls(self, texto):
        """Elimina URLs del texto"""
        return re.sub(r"http\S+|www\S+|https\S+", "", texto, flags=re.MULTILINE)

    def eliminar_emails(self, texto):
        """Elimina emails del texto"""
        return re.sub(r"\S+@\S+", "", texto)

    def eliminar_menciones_hashtags(self, texto):
        """Elimina menciones (@usuario) y hashtags (#tema)"""
        texto = re.sub(r"@\w+", "", texto)
        texto = re.sub(r"#\w+", "", texto)
        return texto

    def eliminar_numeros(self, texto):
        """Elimina números del texto"""
        return re.sub(r"\d+", "", texto)

    def eliminar_caracteres_especiales(self, texto, mantener_espacios=True):
        """
        Elimina caracteres especiales y puntuación
        Mantiene solo letras (incluyendo ñ, acentos) y espacios
        """
        if mantener_espacios:
            # Mantener solo letras, acentos y espacios
            texto = re.sub(r"[^a-záéíóúñüA-ZÁÉÍÓÚÑÜ\s]", "", texto)
        else:
            texto = re.sub(r"[^a-záéíóúñüA-ZÁÉÍÓÚÑÜ]", " ", texto)
        return texto

    def eliminar_espacios_extra(self, texto):
        """Elimina espacios múltiples y limpia el texto"""
        return " ".join(texto.split()).strip()

    def corregir_palabras_comunes(self, texto):
        """Corrige palabras comunes mal escritas"""
        palabras = texto.split()
        palabras_corregidas = [
            self.correcciones.get(palabra, palabra) for palabra in palabras
        ]
        return " ".join(palabras_corregidas)

    def eliminar_palabras_repetidas(self, texto):
        """Elimina palabras que se repiten consecutivamente"""
        palabras = texto.split()
        resultado = []
        anterior = None
        for palabra in palabras:
            if palabra != anterior:
                resultado.append(palabra)
                anterior = palabra
        return " ".join(resultado)

    def limpiar_texto_completo(self, texto):
        """
        Aplica todas las transformaciones de limpieza

        Args:
            texto: texto a limpiar (string)

        Returns:
            texto limpio (string)
        """
        if not isinstance(texto, str):
            return ""

        # 1. Convertir a minúsculas
        texto = self.a_minusculas(texto)

        # 2. Eliminar URLs
        texto = self.eliminar_urls(texto)

        # 3. Eliminar emails
        texto = self.eliminar_emails(texto)

        # 4. Eliminar menciones y hashtags
        texto = self.eliminar_menciones_hashtags(texto)

        # 5. Eliminar números
        texto = self.eliminar_numeros(texto)

        # 6. Corregir palabras comunes (antes de eliminar caracteres especiales)
        texto = self.corregir_palabras_comunes(texto)

        # 7. Eliminar caracteres especiales
        texto = self.eliminar_caracteres_especiales(texto)

        # 8. Eliminar palabras repetidas
        texto = self.eliminar_palabras_repetidas(texto)

        # 9. Eliminar espacios extra
        texto = self.eliminar_espacios_extra(texto)

        return texto


def limpiar_X_train(X_train, verbose=True):
    """
    Limpia todos los textos en X_train

    Args:
        X_train: lista, array, o pandas Series con textos
        verbose: mostrar progreso

    Returns:
        lista con textos limpios
    """
    limpiador = LimpiadorTexto()

    X_train_limpio = []
    total = len(X_train)

    if verbose:
        print("🧹 LIMPIANDO X_TRAIN")
        print("=" * 60)
        print(f"Total de textos: {total}\n")

    for i, texto in enumerate(X_train):
        texto_limpio = limpiador.limpiar_texto_completo(texto)
        X_train_limpio.append(texto_limpio)

        # Mostrar progreso cada 100 textos
        if verbose and (i + 1) % 100 == 0:
            print(f"✓ Procesados {i + 1}/{total} textos ({(i + 1) / total * 100:.1f}%)")

    if verbose:
        print(f"\n✅ Limpieza completada: {total} textos procesados")

        # Estadísticas
        palabras_original = sum(len(str(t).split()) for t in X_train)
        palabras_limpio = sum(len(t.split()) for t in X_train_limpio)

        print("\n📊 ESTADÍSTICAS:")
        print(f"   • Palabras originales: {palabras_original}")
        print(f"   • Palabras después de limpieza: {palabras_limpio}")
        print(f"   • Reducción: {(1 - palabras_limpio / palabras_original) * 100:.1f}%")
        print(f"   • Promedio palabras/texto: {palabras_limpio / total:.2f}")

    return X_train_limpio


def mostrar_ejemplos_limpieza(X_train, X_train_limpio, n=5):
    """
    Muestra ejemplos de textos antes y después de la limpieza

    Args:
        X_train: textos originales
        X_train_limpio: textos limpios
        n: número de ejemplos a mostrar
    """
    print("\n" + "=" * 60)
    print("📝 EJEMPLOS DE LIMPIEZA")
    print("=" * 60)

    for i in range(min(n, len(X_train))):
        print(f"\n[Ejemplo {i + 1}]")
        print(f"ORIGINAL:")
        print(f"  {str(X_train[i])[:100]}{'...' if len(str(X_train[i])) > 100 else ''}")
        print(f"LIMPIO:")
        print(
            f"  {X_train_limpio[i][:100]}{'...' if len(X_train_limpio[i]) > 100 else ''}"
        )


# =============================================================================
# EJEMPLO DE USO
# =============================================================================

if __name__ == "__main__":
    print("\n" + "🚀" * 30)
    print("LIMPIEZA DE X_TRAIN")
    print("🚀" * 30 + "\n")

    # Datos de ejemplo (simula tu X_train)
    X_train = [
        "¡¡¡Este producto es EXCELENTE!!! Lo recomiendo 100% 😊 http://ejemplo.com",
        "Muy mal servicio, NO lo compren!!! números: 12345",
        "Buen producto xq cumple con lo esperado tmb es barato",
        "TERRIBLE experiencia @usuario #hashtag NO funciona",
        "Email: test@ejemplo.com y caracteres $$$ especiales !!!",
        "   Texto   con    espacios    múltiples   repetidos repetidos",
        "jajajaja muy bueno jeje lo recomiendo jajaja",
        "Q bueno k funciona bn tmb es rápido x eso lo compré",
    ]

    print(f"📦 Dataset de ejemplo: {len(X_train)} textos\n")

    # Limpiar X_train
    X_train_limpio = limpiar_X_train(X_train, verbose=True)

    # Mostrar ejemplos
    mostrar_ejemplos_limpieza(X_train, X_train_limpio, n=5)

    print("\n" + "=" * 60)
    print("💾 GUARDAR RESULTADOS")
    print("=" * 60)

    # Opción 1: Guardar como lista de Python
    import pickle

    with open("X_train_limpio.pkl", "wb") as f:
        pickle.dump(X_train_limpio, f)
    print("✓ Guardado en: X_train_limpio.pkl")

    # Opción 2: Guardar como CSV (si tienes pandas)
    try:
        import pandas as pd

        df = pd.DataFrame({"texto_original": X_train, "texto_limpio": X_train_limpio})
        df.to_csv("X_train_limpio.csv", index=False, encoding="utf-8")
        print("✓ Guardado en: X_train_limpio.csv")
    except:
        print("⚠️  pandas no disponible, CSV no guardado")

    # Opción 3: Guardar como texto plano
    with open("X_train_limpio.txt", "w", encoding="utf-8") as f:
        for texto in X_train_limpio:
            f.write(texto + "\n")
    print("✓ Guardado en: X_train_limpio.txt")

    print("\n" + "=" * 60)
    print("📋 INSTRUCCIONES DE USO")
    print("=" * 60)
    print("""
    Para usar con TU X_train:
    
    # Importar
    from limpiar_X_train import limpiar_X_train
    
    # Limpiar (tu X_train puede ser lista, array o Series)
    X_train_limpio = limpiar_X_train(X_train)
    
    # O con más control:
    from limpiar_X_train import LimpiadorTexto
    
    limpiador = LimpiadorTexto()
    X_train_limpio = [limpiador.limpiar_texto_completo(texto) 
                      for texto in X_train]
    
    # Guardar
    import pickle
    with open('X_train_limpio.pkl', 'wb') as f:
        pickle.dump(X_train_limpio, f)
    
    # Cargar después
    with open('X_train_limpio.pkl', 'rb') as f:
        X_train_limpio = pickle.load(f)
    """)

    print("\n✅ Script completado exitosamente!\n")



🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
LIMPIEZA DE X_TRAIN
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

📦 Dataset de ejemplo: 8 textos

🧹 LIMPIANDO X_TRAIN
Total de textos: 8


✅ Limpieza completada: 8 textos procesados

📊 ESTADÍSTICAS:
   • Palabras originales: 65
   • Palabras después de limpieza: 55
   • Reducción: 15.4%
   • Promedio palabras/texto: 6.88

📝 EJEMPLOS DE LIMPIEZA

[Ejemplo 1]
ORIGINAL:
  ¡¡¡Este producto es EXCELENTE!!! Lo recomiendo 100% 😊 http://ejemplo.com
LIMPIO:
  este producto es excelente lo recomiendo

[Ejemplo 2]
ORIGINAL:
  Muy mal servicio, NO lo compren!!! números: 12345
LIMPIO:
  muy mal servicio no lo compren números

[Ejemplo 3]
ORIGINAL:
  Buen producto xq cumple con lo esperado tmb es barato
LIMPIO:
  buen producto porque cumple con lo esperado también es barato

[Ejemplo 4]
ORIGINAL:
  TERRIBLE experiencia @usuario #hashtag NO funciona
LIMPIO:
  terrible experiencia no funciona

[Ejemplo 5]
ORIGINAL:
  Email: test@ejemplo.com y caracteres $$$ especiales !!!
LIMPIO:
  e

## 2) Representacion del texto

Luego de tener los datos limpios, realizar la representación de los datos de texto para poder usarse en modelos de machine learning.

In [18]:
"""
SCRIPT PARA TOKENIZACIÓN + LEMATIZACIÓN + REPRESENTACIÓN
==========================================================

Aplica a X_train_limpio:
1. ✅ Tokenización (dividir en palabras)
2. ✅ Lematización (reducir a forma base) o Stemming
3. ✅ Representación numérica (Bag of Words o TF-IDF)

USO DIRECTO:
------------
>>> from procesar_X_train import procesar_X_train_tfidf, procesar_X_train_bow
>>> X_train_procesado = procesar_X_train_tfidf(X_train_limpio)
"""

import re
import numpy as np
from collections import Counter
import pickle


# =============================================================================
# CLASE 1: TOKENIZADOR
# =============================================================================


class Tokenizador:
    """Divide texto en tokens (palabras)"""

    def tokenizar(self, texto):
        """Tokeniza un texto en palabras"""
        return texto.split()

    def tokenizar_batch(self, textos):
        """Tokeniza múltiples textos"""
        return [self.tokenizar(texto) for texto in textos]


# =============================================================================
# CLASE 2: STEMMER PARA ESPAÑOL
# =============================================================================


class StemmerEspanol:
    """Stemmer para español - corta palabras a su raíz"""

    def __init__(self):
        self.sufijos = [
            "ísimos",
            "ísimas",
            "amiento",
            "imientos",
            "amiento",
            "aciones",
            "ución",
            "uciones",
            "adores",
            "adoras",
            "ísimo",
            "ísima",
            "mente",
            "ancia",
            "encia",
            "ación",
            "ición",
            "ador",
            "adora",
            "ante",
            "ando",
            "iendo",
            "aron",
            "ieron",
            "ado",
            "ido",
            "able",
            "ible",
            "osos",
            "osas",
            "ivo",
            "iva",
            "dad",
            "tad",
            "eza",
            "miento",
            "es",
            "os",
            "as",
            "a",
            "o",
            "e",
        ]
        self.min_length = 3

    def stem(self, palabra):
        """Reduce una palabra a su raíz"""
        if len(palabra) <= self.min_length:
            return palabra

        for sufijo in self.sufijos:
            if (
                palabra.endswith(sufijo)
                and len(palabra) - len(sufijo) >= self.min_length
            ):
                return palabra[: -len(sufijo)]

        return palabra

    def stem_tokens(self, tokens):
        """Aplica stemming a lista de tokens"""
        return [self.stem(token) for token in tokens]


# =============================================================================
# CLASE 3: LEMATIZADOR PARA ESPAÑOL
# =============================================================================


class LematizadorEspanol:
    """Lematizador para español - convierte a forma canónica"""

    def __init__(self):
        self.diccionario_lemas = {
            # Verbos ser/estar
            "soy": "ser",
            "eres": "ser",
            "es": "ser",
            "somos": "ser",
            "son": "ser",
            "fui": "ser",
            "fue": "ser",
            "fueron": "ser",
            "siendo": "ser",
            "sido": "ser",
            "estoy": "estar",
            "estás": "estar",
            "está": "estar",
            "estamos": "estar",
            "están": "estar",
            "estuve": "estar",
            "estuvo": "estar",
            "estado": "estar",
            # Verbo tener
            "tengo": "tener",
            "tienes": "tener",
            "tiene": "tener",
            "tenemos": "tener",
            "tienen": "tener",
            "tuve": "tener",
            "tuvo": "tener",
            "tenido": "tener",
            # Verbo hacer
            "hago": "hacer",
            "haces": "hacer",
            "hace": "hacer",
            "hacemos": "hacer",
            "hacen": "hacer",
            "hice": "hacer",
            "hizo": "hacer",
            "hecho": "hacer",
            # Verbo ir
            "voy": "ir",
            "vas": "ir",
            "va": "ir",
            "vamos": "ir",
            "van": "ir",
            "ido": "ir",
            # Plurales comunes
            "datos": "dato",
            "textos": "texto",
            "palabras": "palabra",
            "modelos": "modelo",
            "redes": "red",
            "árboles": "árbol",
        }

    def lematizar(self, palabra):
        """Obtiene el lema de una palabra"""
        if palabra in self.diccionario_lemas:
            return self.diccionario_lemas[palabra]

        # Reglas simples para plurales
        if palabra.endswith("es") and len(palabra) > 4:
            return palabra[:-2]
        elif palabra.endswith("s") and len(palabra) > 3:
            return palabra[:-1]

        return palabra

    def lematizar_tokens(self, tokens):
        """Aplica lematización a lista de tokens"""
        return [self.lematizar(token) for token in tokens]


# =============================================================================
# CLASE 4: REPRESENTADOR (BAG OF WORDS Y TF-IDF)
# =============================================================================


class RepresentadorTexto:
    """
    Convierte textos en vectores numéricos después de tokenizar y lematizar
    """

    def __init__(
        self, usar_lematizacion=True, usar_stemming=False, max_features=1000, min_df=1
    ):
        """
        Args:
            usar_lematizacion: si usar lematización
            usar_stemming: si usar stemming
            max_features: número máximo de palabras en vocabulario
            min_df: frecuencia mínima de palabra
        """
        self.usar_lematizacion = usar_lematizacion
        self.usar_stemming = usar_stemming
        self.max_features = max_features
        self.min_df = min_df

        self.tokenizador = Tokenizador()
        self.lematizador = LematizadorEspanol() if usar_lematizacion else None
        self.stemmer = StemmerEspanol() if usar_stemming else None

        self.vocabulario = {}
        self.feature_names = []
        self.idf_dict = {}

    def _procesar_texto(self, texto):
        """Tokeniza y aplica lematización o stemming"""
        # Tokenizar
        tokens = self.tokenizador.tokenizar(texto)

        # Lematizar o hacer stemming
        if self.usar_lematizacion:
            tokens = self.lematizador.lematizar_tokens(tokens)
        elif self.usar_stemming:
            tokens = self.stemmer.stem_tokens(tokens)

        return tokens

    def fit(self, textos):
        """Construye vocabulario a partir de textos"""
        # Procesar todos los textos
        todos_tokens = []
        for texto in textos:
            tokens = self._procesar_texto(texto)
            todos_tokens.append(tokens)

        # Contar frecuencia de documentos
        doc_freq = Counter()
        for tokens in todos_tokens:
            tokens_unicos = set(tokens)
            doc_freq.update(tokens_unicos)

        # Filtrar por frecuencia mínima
        tokens_validos = [
            token for token, freq in doc_freq.items() if freq >= self.min_df
        ]

        # Limitar features
        if self.max_features and len(tokens_validos) > self.max_features:
            tokens_ordenados = sorted(
                doc_freq.items(), key=lambda x: x[1], reverse=True
            )
            tokens_validos = [t[0] for t in tokens_ordenados[: self.max_features]]

        # Crear vocabulario
        self.vocabulario = {token: idx for idx, token in enumerate(tokens_validos)}
        self.feature_names = tokens_validos

        # Calcular IDF
        n_docs = len(textos)
        for token in tokens_validos:
            df = doc_freq[token]
            self.idf_dict[token] = np.log(n_docs / df) + 1

        return self

    def transform_bow(self, textos):
        """Transforma textos a Bag of Words"""
        X = np.zeros((len(textos), len(self.vocabulario)))

        for i, texto in enumerate(textos):
            tokens = self._procesar_texto(texto)
            token_counts = Counter(tokens)

            for token, count in token_counts.items():
                if token in self.vocabulario:
                    idx = self.vocabulario[token]
                    X[i, idx] = count

        return X

    def transform_tfidf(self, textos):
        """Transforma textos a TF-IDF"""
        X = np.zeros((len(textos), len(self.vocabulario)))

        for i, texto in enumerate(textos):
            tokens = self._procesar_texto(texto)
            token_counts = Counter(tokens)

            # Calcular TF-IDF
            for token, tf in token_counts.items():
                if token in self.vocabulario:
                    idx = self.vocabulario[token]
                    idf = self.idf_dict[token]
                    X[i, idx] = tf * idf

        # Normalización L2
        normas = np.sqrt(np.sum(X**2, axis=1, keepdims=True))
        normas[normas == 0] = 1
        X = X / normas

        return X

    def fit_transform_bow(self, textos):
        """Ajusta y transforma a BoW"""
        self.fit(textos)
        return self.transform_bow(textos)

    def fit_transform_tfidf(self, textos):
        """Ajusta y transforma a TF-IDF"""
        self.fit(textos)
        return self.transform_tfidf(textos)


# =============================================================================
# FUNCIONES DE USO DIRECTO
# =============================================================================


def procesar_X_train_tfidf(
    X_train_limpio, max_features=1000, min_df=2, usar_lematizacion=True, verbose=True
):
    """
    Procesa X_train_limpio y genera representación TF-IDF

    Args:
        X_train_limpio: lista de textos ya limpios
        max_features: número máximo de palabras
        min_df: frecuencia mínima de palabra
        usar_lematizacion: True para lematización, False para stemming
        verbose: mostrar progreso

    Returns:
        X_tfidf: matriz TF-IDF (numpy array)
        representador: objeto entrenado (para transformar X_test después)
    """
    if verbose:
        print("🔹 PROCESANDO X_TRAIN CON TF-IDF")
        print("=" * 60)
        print(f"Total de textos: {len(X_train_limpio)}")
        print(f"Max features: {max_features}")
        print(f"Min df: {min_df}")
        print(f"Método: {'Lematización' if usar_lematizacion else 'Stemming'}\n")

    # Crear representador
    representador = RepresentadorTexto(
        usar_lematizacion=usar_lematizacion,
        usar_stemming=not usar_lematizacion,
        max_features=max_features,
        min_df=min_df,
    )

    # Procesar
    X_tfidf = representador.fit_transform_tfidf(X_train_limpio)

    if verbose:
        print(f"✅ Procesamiento completado")
        print(f"\n📊 RESULTADOS:")
        print(f"   • Forma de la matriz: {X_tfidf.shape}")
        print(f"   • Vocabulario: {len(representador.vocabulario)} palabras")
        print(f"   • Sparsity: {(X_tfidf == 0).sum() / X_tfidf.size * 100:.2f}%")

        print(f"\n📝 Top 10 palabras del vocabulario:")
        for i, palabra in enumerate(representador.feature_names[:10], 1):
            print(f"   {i:2d}. {palabra}")

    return X_tfidf, representador


def procesar_X_train_bow(
    X_train_limpio, max_features=1000, min_df=2, usar_lematizacion=True, verbose=True
):
    """
    Procesa X_train_limpio y genera representación Bag of Words

    Args:
        X_train_limpio: lista de textos ya limpios
        max_features: número máximo de palabras
        min_df: frecuencia mínima de palabra
        usar_lematizacion: True para lematización, False para stemming
        verbose: mostrar progreso

    Returns:
        X_bow: matriz Bag of Words (numpy array)
        representador: objeto entrenado (para transformar X_test después)
    """
    if verbose:
        print("🔹 PROCESANDO X_TRAIN CON BAG OF WORDS")
        print("=" * 60)
        print(f"Total de textos: {len(X_train_limpio)}")
        print(f"Max features: {max_features}")
        print(f"Min df: {min_df}")
        print(f"Método: {'Lematización' if usar_lematizacion else 'Stemming'}\n")

    # Crear representador
    representador = RepresentadorTexto(
        usar_lematizacion=usar_lematizacion,
        usar_stemming=not usar_lematizacion,
        max_features=max_features,
        min_df=min_df,
    )

    # Procesar
    X_bow = representador.fit_transform_bow(X_train_limpio)

    if verbose:
        print(f"✅ Procesamiento completado")
        print(f"\n📊 RESULTADOS:")
        print(f"   • Forma de la matriz: {X_bow.shape}")
        print(f"   • Vocabulario: {len(representador.vocabulario)} palabras")
        print(f"   • Valores totales: {X_bow.sum():.0f}")

        print(f"\n📝 Top 10 palabras del vocabulario:")
        for i, palabra in enumerate(representador.feature_names[:10], 1):
            print(f"   {i:2d}. {palabra}")

    return X_bow, representador


def procesar_X_test(X_test_limpio, representador, metodo="tfidf", verbose=True):
    """
    Procesa X_test usando el mismo representador de X_train

    Args:
        X_test_limpio: lista de textos ya limpios
        representador: objeto RepresentadorTexto ya entrenado
        metodo: 'tfidf' o 'bow'
        verbose: mostrar progreso

    Returns:
        X_test_procesado: matriz procesada
    """
    if verbose:
        print(f"\n🔹 PROCESANDO X_TEST CON {metodo.upper()}")
        print("=" * 60)

    if metodo == "tfidf":
        X_test_procesado = representador.transform_tfidf(X_test_limpio)
    else:
        X_test_procesado = representador.transform_bow(X_test_limpio)

    if verbose:
        print(f"✅ X_test procesado: {X_test_procesado.shape}")

    return X_test_procesado


def guardar_representador(representador, filename="representador.pkl"):
    """Guarda el representador entrenado"""
    with open(filename, "wb") as f:
        pickle.dump(representador, f)
    print(f"✅ Representador guardado: {filename}")


def cargar_representador(filename="representador.pkl"):
    """Carga un representador guardado"""
    with open(filename, "rb") as f:
        representador = pickle.load(f)
    print(f"✅ Representador cargado: {filename}")
    return representador


# =============================================================================
# EJEMPLO DE USO
# =============================================================================

if __name__ == "__main__":
    print("\n" + "🚀" * 30)
    print("TOKENIZACIÓN + LEMATIZACIÓN + REPRESENTACIÓN")
    print("🚀" * 30 + "\n")

    # Simular X_train_limpio (textos ya limpios)
    X_train_limpio = [
        "machine learning es fascinante",
        "python es genial para datos",
        "deep learning utiliza redes neuronales",
        "procesamiento de lenguaje natural",
        "algoritmos de clasificación y regresión",
        "modelos de machine learning",
        "entrenamiento de redes neuronales",
        "análisis de datos con python",
        "inteligencia artificial y machine learning",
        "procesamiento de texto con python",
    ]

    print(f"📦 Dataset: {len(X_train_limpio)} textos\n")

    # OPCIÓN 1: TF-IDF con Lematización
    print("\n" + "=" * 60)
    print("OPCIÓN 1: TF-IDF CON LEMATIZACIÓN")
    print("=" * 60)

    X_tfidf, representador_tfidf = procesar_X_train_tfidf(
        X_train_limpio, max_features=50, min_df=1, usar_lematizacion=True, verbose=True
    )

    # OPCIÓN 2: Bag of Words con Stemming
    print("\n\n" + "=" * 60)
    print("OPCIÓN 2: BAG OF WORDS CON STEMMING")
    print("=" * 60)

    X_bow, representador_bow = procesar_X_train_bow(
        X_train_limpio,
        max_features=50,
        min_df=1,
        usar_lematizacion=False,  # Usar stemming
        verbose=True,
    )

    # Guardar representadores
    print("\n" + "=" * 60)
    print("GUARDAR REPRESENTADORES")
    print("=" * 60)

    guardar_representador(representador_tfidf, "representador_tfidf.pkl")
    guardar_representador(representador_bow, "representador_bow.pkl")

    # Ejemplo con X_test
    print("\n" + "=" * 60)
    print("PROCESAR X_TEST")
    print("=" * 60)

    X_test_limpio = ["machine learning con python", "redes neuronales profundas"]

    X_test_tfidf = procesar_X_test(
        X_test_limpio, representador_tfidf, metodo="tfidf", verbose=True
    )

    # Instrucciones
    print("\n" + "=" * 60)
    print("📋 CÓMO USAR CON TU X_TRAIN")
    print("=" * 60)
    print("""
    # 1. Importar
    from procesar_X_train import procesar_X_train_tfidf, procesar_X_test
    
    # 2. Procesar X_train_limpio
    X_train_tfidf, representador = procesar_X_train_tfidf(
        X_train_limpio,
        max_features=1000,
        min_df=2,
        usar_lematizacion=True
    )
    
    # 3. Procesar X_test_limpio
    X_test_tfidf = procesar_X_test(
        X_test_limpio,
        representador,
        metodo='tfidf'
    )
    
    # 4. Entrenar modelo
    from sklearn.linear_model import LogisticRegression
    modelo = LogisticRegression()
    modelo.fit(X_train_tfidf, y_train)
    
    # 5. Evaluar
    accuracy = modelo.score(X_test_tfidf, y_test)
    print(f"Accuracy: {accuracy:.2%}")
    """)

    print("\n✅ Script completado exitosamente!\n")



🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
TOKENIZACIÓN + LEMATIZACIÓN + REPRESENTACIÓN
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

📦 Dataset: 10 textos


OPCIÓN 1: TF-IDF CON LEMATIZACIÓN
🔹 PROCESANDO X_TRAIN CON TF-IDF
Total de textos: 10
Max features: 50
Min df: 1
Método: Lematización

✅ Procesamiento completado

📊 RESULTADOS:
   • Forma de la matriz: (10, 27)
   • Vocabulario: 27 palabras
   • Sparsity: 82.96%

📝 Top 10 palabras del vocabulario:
    1. learning
    2. fascinante
    3. machine
    4. ser
    5. para
    6. dato
    7. python
    8. genial
    9. red
   10. deep


OPCIÓN 2: BAG OF WORDS CON STEMMING
🔹 PROCESANDO X_TRAIN CON BAG OF WORDS
Total de textos: 10
Max features: 50
Min df: 1
Método: Stemming

✅ Procesamiento completado

📊 RESULTADOS:
   • Forma de la matriz: (10, 27)
   • Vocabulario: 27 palabras
   • Valores totales: 46

📝 Top 10 palabras del vocabulario:
    1. learning
    2. es
    3. fascin
    4. machin
    5. dat
    6. python
    7. par
    8. genial
    9. red
   10. deep

## 3) Entrenar un modelo de machine learning de clasificación

Utilizar un modelo de clasificación para entrenar y evaluar el modelo con los datos preparados.


In [20]:
"""
ENTRENAR MODELO DE CLASIFICACIÓN
=================================

Script simple para entrenar un modelo de clasificación
con los datos ya preparados

USO:
----
>>> from entrenar_modelo import entrenar_modelo
>>> modelo = entrenar_modelo(X_train_procesado, y_train)
"""

import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

warnings.filterwarnings("ignore")


def entrenar_modelo(
    X_train, y_train, modelo_tipo="logistic", test_size=0.2, verbose=True
):
    """
    Entrena un modelo de clasificación

    Args:
        X_train: matriz de características (ya procesada con TF-IDF o BoW)
        y_train: etiquetas
        modelo_tipo: tipo de modelo a entrenar
            - 'logistic': Logistic Regression (RECOMENDADO)
            - 'random_forest': Random Forest
            - 'svm': Support Vector Machine
            - 'decision_tree': Decision Tree
            - 'knn': K-Nearest Neighbors
            - 'gradient_boosting': Gradient Boosting
        test_size: proporción para validación (0.2 = 20%)
        verbose: mostrar información

    Returns:
        modelo entrenado
    """

    if verbose:
        print("🤖 ENTRENANDO MODELO DE CLASIFICACIÓN")
        print("=" * 60)
        print(f"Tipo de modelo: {modelo_tipo}")
        print(f"Total de muestras: {len(X_train)}")
        print(f"Características: {X_train.shape[1]}")
        print(f"Clases: {np.unique(y_train)}\n")

    # Dividir en train/validación
    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_train, y_train, test_size=test_size, random_state=42, stratify=y_train
    )

    if verbose:
        print(f"Train: {X_train_split.shape[0]} muestras")
        print(f"Validación: {X_val.shape[0]} muestras\n")

    # Seleccionar modelo
    if modelo_tipo == "logistic":
        modelo = LogisticRegression(max_iter=1000, random_state=42)

    elif modelo_tipo == "random_forest":
        modelo = RandomForestClassifier(n_estimators=100, random_state=42)

    elif modelo_tipo == "svm":
        modelo = SVC(kernel="linear", probability=True, random_state=42)

    elif modelo_tipo == "decision_tree":
        modelo = DecisionTreeClassifier(max_depth=10, random_state=42)

    elif modelo_tipo == "knn":
        modelo = KNeighborsClassifier(n_neighbors=5)

    elif modelo_tipo == "gradient_boosting":
        modelo = GradientBoostingClassifier(n_estimators=100, random_state=42)

    else:
        raise ValueError(f"Modelo '{modelo_tipo}' no soportado")

    # Entrenar
    if verbose:
        print(f"Entrenando {modelo_tipo}...")

    modelo.fit(X_train_split, y_train_split)

    if verbose:
        print("✅ Modelo entrenado\n")

    # Evaluar en validación
    y_val_pred = modelo.predict(X_val)
    accuracy = accuracy_score(y_val, y_val_pred)

    if verbose:
        print("📊 RESULTADOS EN VALIDACIÓN:")
        print(f"   Accuracy: {accuracy:.4f}\n")
        print(classification_report(y_val, y_val_pred, zero_division=0))

    # Re-entrenar con todos los datos
    if verbose:
        print("\n🔄 Re-entrenando con todos los datos...")

    modelo.fit(X_train, y_train)

    if verbose:
        print("✅ Modelo final entrenado con todos los datos\n")

    return modelo


def guardar_modelo(modelo, filename="modelo.pkl"):
    """
    Guarda el modelo entrenado

    Args:
        modelo: modelo entrenado
        filename: nombre del archivo
    """
    with open(filename, "wb") as f:
        pickle.dump(modelo, f)
    print(f"✅ Modelo guardado: {filename}")


def cargar_modelo(filename="modelo.pkl"):
    """
    Carga un modelo guardado

    Args:
        filename: nombre del archivo

    Returns:
        modelo cargado
    """
    with open(filename, "rb") as f:
        modelo = pickle.load(f)
    print(f"✅ Modelo cargado: {filename}")
    return modelo


def predecir(modelo, X_nuevos):
    """
    Predice con el modelo entrenado

    Args:
        modelo: modelo entrenado
        X_nuevos: datos nuevos (ya procesados)

    Returns:
        predicciones
    """
    return modelo.predict(X_nuevos)


# =============================================================================
# EJEMPLO DE USO
# =============================================================================

if __name__ == "__main__":
    print("\n" + "🚀" * 30)
    print("ENTRENAR MODELO DE CLASIFICACIÓN - EJEMPLO")
    print("🚀" * 30 + "\n")

    # Generar datos de ejemplo
    from sklearn.datasets import make_classification

    X, y = make_classification(
        n_samples=200, n_features=50, n_informative=30, n_classes=2, random_state=42
    )

    print("📦 Datos de ejemplo generados")
    print(f"   Forma: {X.shape}")
    print(f"   Clases: {np.unique(y)}\n")

    # Entrenar modelo
    modelo = entrenar_modelo(X, y, modelo_tipo="logistic", test_size=0.2, verbose=True)

    # Guardar modelo
    print("=" * 60)
    print("💾 GUARDANDO MODELO")
    print("=" * 60)
    guardar_modelo(modelo, "mi_modelo.pkl")

    # Cargar y usar
    print("\n" + "=" * 60)
    print("🔮 CARGAR Y PREDECIR")
    print("=" * 60)

    modelo_cargado = cargar_modelo("mi_modelo.pkl")

    # Predecir con nuevos datos
    X_nuevos = X[:5]
    predicciones = predecir(modelo_cargado, X_nuevos)

    print(f"\n✅ Predicciones: {predicciones}")
    print(f"   Reales:       {y[:5]}")

    # Instrucciones de uso
    print("\n" + "=" * 60)
    print("📋 CÓMO USAR CON TUS DATOS")
    print("=" * 60)
    print("""
    # ========================================
    # PASO 1: Entrenar con tus datos
    # ========================================
    from entrenar_modelo import entrenar_modelo, guardar_modelo
    
    # Tu X_train ya procesado (matriz TF-IDF o BoW)
    # Tu y_train (etiquetas)
    
    modelo = entrenar_modelo(
        X_train_procesado,
        y_train,
        modelo_tipo='logistic',  # o 'random_forest', 'svm', etc.
        test_size=0.2
    )
    
    guardar_modelo(modelo, 'mi_modelo_final.pkl')
    
    
    # ========================================
    # PASO 2: Usar en producción
    # ========================================
    from entrenar_modelo import cargar_modelo, predecir
    
    # Cargar modelo
    modelo = cargar_modelo('mi_modelo_final.pkl')
    
    # Predecir (X_nuevos debe estar procesado igual que X_train)
    predicciones = predecir(modelo, X_nuevos_procesados)
    
    print(predicciones)
    """)

    print("\n✅ Ejemplo completado!\n")



🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
ENTRENAR MODELO DE CLASIFICACIÓN - EJEMPLO
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

📦 Datos de ejemplo generados
   Forma: (200, 50)
   Clases: [0 1]

🤖 ENTRENANDO MODELO DE CLASIFICACIÓN
Tipo de modelo: logistic
Total de muestras: 200
Características: 50
Clases: [0 1]

Train: 160 muestras
Validación: 40 muestras

Entrenando logistic...
✅ Modelo entrenado

📊 RESULTADOS EN VALIDACIÓN:
   Accuracy: 0.7250

              precision    recall  f1-score   support

           0       0.68      0.79      0.73        19
           1       0.78      0.67      0.72        21

    accuracy                           0.72        40
   macro avg       0.73      0.73      0.72        40
weighted avg       0.73      0.72      0.72        40


🔄 Re-entrenando con todos los datos...
✅ Modelo final entrenado con todos los datos

💾 GUARDANDO MODELO
✅ Modelo guardado: mi_modelo.pkl

🔮 CARGAR Y PREDECIR
✅ Modelo cargado: mi_modelo.pkl

✅ Predicciones: [1 1 0 0 1]
   Reales:       [1 1 

### Evaluar el modelo con los datos de prueba

Usar el modelo para predecir en `X_test` y evaluar con `y_test`

**Nota:** Recuerde que `X_test` debe pasar por los mismos procesos de limpieza y representación que `X_train`

In [21]:
"""
SCRIPT TODO-EN-UNO COMPLETO
============================

Este script contiene TODO lo necesario en un solo archivo:
1. Limpieza
2. Tokenización + Lematización
3. Representación (TF-IDF)
4. Entrenamiento
5. Evaluación

NO necesita importar otros scripts.
Solo reemplaza tus datos en el PASO 1.

USO:
----
1. Reemplaza X_train, y_train, X_test, y_test con tus datos
2. Ejecuta: python script_completo_todo_en_uno.py
"""

import re
import numpy as np
import pickle
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings

warnings.filterwarnings("ignore")


# =============================================================================
# CLASE 1: LIMPIADOR DE TEXTO
# =============================================================================


class LimpiadorTexto:
    """Limpia texto: minúsculas, sin caracteres especiales, etc."""

    def __init__(self):
        self.correcciones = {
            "q": "que",
            "xq": "porque",
            "pq": "porque",
            "tmb": "también",
            "tb": "también",
            "x": "por",
            "d": "de",
            "k": "que",
            "bn": "bien",
            "jajaja": "jaja",
            "jeje": "jaja",
        }

    def limpiar(self, texto):
        """Limpia un texto"""
        if not isinstance(texto, str):
            return ""

        texto = texto.lower()
        texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)
        texto = re.sub(r"\S+@\S+", "", texto)
        texto = re.sub(r"@\w+", "", texto)
        texto = re.sub(r"#\w+", "", texto)
        texto = re.sub(r"\d+", "", texto)

        # Corregir palabras
        palabras = texto.split()
        palabras = [self.correcciones.get(p, p) for p in palabras]
        texto = " ".join(palabras)

        texto = re.sub(r"[^a-záéíóúñüA-ZÁÉÍÓÚÑÜ\s]", "", texto)
        texto = " ".join(texto.split()).strip()

        return texto


# =============================================================================
# CLASE 2: LEMATIZADOR
# =============================================================================


class Lematizador:
    """Lematiza palabras (reduce a forma base)"""

    def __init__(self):
        self.lemas = {
            "soy": "ser",
            "eres": "ser",
            "es": "ser",
            "somos": "ser",
            "son": "ser",
            "estoy": "estar",
            "estás": "estar",
            "está": "estar",
            "están": "estar",
            "tengo": "tener",
            "tienes": "tener",
            "tiene": "tener",
            "tienen": "tener",
            "datos": "dato",
            "textos": "texto",
            "palabras": "palabra",
            "modelos": "modelo",
            "redes": "red",
        }

    def lematizar(self, palabra):
        if palabra in self.lemas:
            return self.lemas[palabra]
        if palabra.endswith("es") and len(palabra) > 4:
            return palabra[:-2]
        elif palabra.endswith("s") and len(palabra) > 3:
            return palabra[:-1]
        return palabra


# =============================================================================
# CLASE 3: REPRESENTADOR TF-IDF
# =============================================================================


class RepresentadorTFIDF:
    """Tokeniza + Lematiza + Representa como TF-IDF"""

    def __init__(self, max_features=1000, min_df=2):
        self.max_features = max_features
        self.min_df = min_df
        self.lematizador = Lematizador()
        self.vocabulario = {}
        self.feature_names = []
        self.idf_dict = {}

    def _procesar(self, texto):
        """Tokeniza y lematiza"""
        tokens = texto.split()
        return [self.lematizador.lematizar(t) for t in tokens]

    def fit(self, textos):
        """Construye vocabulario"""
        todos_tokens = [self._procesar(t) for t in textos]

        doc_freq = Counter()
        for tokens in todos_tokens:
            doc_freq.update(set(tokens))

        tokens_validos = [t for t, f in doc_freq.items() if f >= self.min_df]

        if len(tokens_validos) > self.max_features:
            tokens_ordenados = sorted(
                doc_freq.items(), key=lambda x: x[1], reverse=True
            )
            tokens_validos = [t[0] for t in tokens_ordenados[: self.max_features]]

        self.vocabulario = {t: i for i, t in enumerate(tokens_validos)}
        self.feature_names = tokens_validos

        n_docs = len(textos)
        for token in tokens_validos:
            df = doc_freq[token]
            self.idf_dict[token] = np.log(n_docs / df) + 1

        return self

    def transform(self, textos):
        """Transforma textos a TF-IDF"""
        X = np.zeros((len(textos), len(self.vocabulario)))

        for i, texto in enumerate(textos):
            tokens = self._procesar(texto)
            token_counts = Counter(tokens)

            for token, tf in token_counts.items():
                if token in self.vocabulario:
                    idx = self.vocabulario[token]
                    idf = self.idf_dict[token]
                    X[i, idx] = tf * idf

        # Normalización L2
        normas = np.sqrt(np.sum(X**2, axis=1, keepdims=True))
        normas[normas == 0] = 1
        X = X / normas

        return X

    def fit_transform(self, textos):
        self.fit(textos)
        return self.transform(textos)


# =============================================================================
# FUNCIÓN: ENTRENAR MODELO
# =============================================================================


def entrenar_modelo(X_train, y_train, modelo_tipo="logistic", verbose=True):
    """Entrena modelo de clasificación"""

    if verbose:
        print("🤖 ENTRENANDO MODELO")
        print("=" * 60)
        print(f"Tipo: {modelo_tipo}")
        print(f"Muestras: {len(X_train)}")

    if modelo_tipo == "logistic":
        modelo = LogisticRegression(max_iter=1000, random_state=42)
    else:
        modelo = LogisticRegression(max_iter=1000, random_state=42)

    modelo.fit(X_train, y_train)

    if verbose:
        print("✅ Modelo entrenado\n")

    return modelo


# =============================================================================
# FUNCIÓN: EVALUAR MODELO
# =============================================================================


def evaluar_modelo(modelo, X_test, y_test, verbose=True):
    """Evalúa el modelo"""

    if verbose:
        print("🔍 EVALUANDO MODELO")
        print("=" * 60)

    y_pred = modelo.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    if verbose:
        print(f"\n✅ ACCURACY: {accuracy:.4f} ({accuracy * 100:.2f}%)\n")

        print("📊 MATRIZ DE CONFUSIÓN:")
        cm = confusion_matrix(y_test, y_pred)
        print(cm)

        print("\n📈 REPORTE DETALLADO:")
        print(classification_report(y_test, y_pred, zero_division=0))

    return {"accuracy": accuracy, "y_pred": y_pred, "confusion_matrix": cm}


# =============================================================================
# PIPELINE COMPLETO
# =============================================================================


def pipeline_completo(
    X_train_crudo, y_train, X_test_crudo, y_test, max_features=1000, min_df=2
):
    """
    Pipeline completo de inicio a fin

    Args:
        X_train_crudo: textos de entrenamiento (sin procesar)
        y_train: etiquetas de entrenamiento
        X_test_crudo: textos de prueba (sin procesar)
        y_test: etiquetas de prueba
        max_features: número máximo de palabras
        min_df: frecuencia mínima de palabra

    Returns:
        modelo entrenado, representador, resultados
    """

    print("\n" + "🚀" * 35)
    print("PIPELINE COMPLETO - CLASIFICACIÓN DE TEXTO")
    print("🚀" * 35 + "\n")

    # PASO 1: Limpiar
    print("=" * 60)
    print("PASO 1: LIMPIEZA")
    print("=" * 60)

    limpiador = LimpiadorTexto()

    print("Limpiando X_train...")
    X_train_limpio = [limpiador.limpiar(texto) for texto in X_train_crudo]
    print(f"✓ {len(X_train_limpio)} textos limpios")

    print("Limpiando X_test...")
    X_test_limpio = [limpiador.limpiar(texto) for texto in X_test_crudo]
    print(f"✓ {len(X_test_limpio)} textos limpios\n")

    # Mostrar ejemplo
    print(f"Ejemplo de limpieza:")
    print(f"  Original: {X_train_crudo[0][:60]}...")
    print(f"  Limpio:   {X_train_limpio[0]}\n")

    # PASO 2: Procesar y representar
    print("=" * 60)
    print("PASO 2: TOKENIZACIÓN + LEMATIZACIÓN + TF-IDF")
    print("=" * 60)

    representador = RepresentadorTFIDF(max_features=max_features, min_df=min_df)

    print("Procesando X_train...")
    X_train_procesado = representador.fit_transform(X_train_limpio)
    print(f"✓ Matriz TF-IDF: {X_train_procesado.shape}")
    print(f"✓ Vocabulario: {len(representador.vocabulario)} palabras")

    print("\nProcesando X_test con el MISMO representador...")
    X_test_procesado = representador.transform(X_test_limpio)
    print(f"✓ Matriz TF-IDF: {X_test_procesado.shape}\n")

    # PASO 3: Entrenar
    print("=" * 60)
    print("PASO 3: ENTRENAMIENTO")
    print("=" * 60)

    modelo = entrenar_modelo(X_train_procesado, y_train, verbose=True)

    # PASO 4: Evaluar
    print("=" * 60)
    print("PASO 4: EVALUACIÓN CON X_TEST")
    print("=" * 60)

    resultados = evaluar_modelo(modelo, X_test_procesado, y_test, verbose=True)

    # PASO 5: Mostrar predicciones
    print("=" * 60)
    print("🔍 PREDICCIONES vs REALES")
    print("=" * 60)

    y_pred = resultados["y_pred"]
    for i, (texto, real, pred) in enumerate(
        zip(X_test_crudo[:10], y_test[:10], y_pred[:10]), 1
    ):
        estado = "✅" if real == pred else "❌"
        print(f"\n{i}. {estado} Real: {real} | Predicho: {pred}")
        print(f"   Texto: {texto[:60]}...")

    # PASO 6: Guardar
    print("\n" + "=" * 60)
    print("💾 GUARDANDO MODELO Y REPRESENTADOR")
    print("=" * 60)

    with open("modelo_final.pkl", "wb") as f:
        pickle.dump(modelo, f)
    print("✓ Modelo guardado: modelo_final.pkl")

    with open("representador_final.pkl", "wb") as f:
        pickle.dump(representador, f)
    print("✓ Representador guardado: representador_final.pkl")

    # Resumen
    print("\n" + "=" * 60)
    print("📊 RESUMEN")
    print("=" * 60)
    print(f"""
    ✅ X_train: {len(X_train_crudo)} muestras
    ✅ X_test:  {len(X_test_crudo)} muestras
    ✅ Vocabulario: {len(representador.vocabulario)} palabras
    ✅ Accuracy: {resultados["accuracy"]:.2%}
    
    📂 Archivos generados:
       • modelo_final.pkl
       • representador_final.pkl
    """)

    return modelo, representador, resultados


# =============================================================================
# FUNCIÓN: PREDECIR TEXTO NUEVO
# =============================================================================


def predecir_texto_nuevo(texto_crudo, modelo, representador):
    """
    Predice la clase de un texto nuevo

    Args:
        texto_crudo: texto sin procesar
        modelo: modelo entrenado
        representador: representador entrenado

    Returns:
        predicción
    """
    # Limpiar
    limpiador = LimpiadorTexto()
    texto_limpio = limpiador.limpiar(texto_crudo)

    # Representar
    X_procesado = representador.transform([texto_limpio])

    # Predecir
    prediccion = modelo.predict(X_procesado)[0]

    # Probabilidad
    if hasattr(modelo, "predict_proba"):
        probabilidad = modelo.predict_proba(X_procesado)[0]
        confianza = probabilidad[prediccion]
        return prediccion, confianza

    return prediccion, None


# =============================================================================
# MAIN: EJEMPLO DE USO
# =============================================================================

if __name__ == "__main__":
    # =========================================================================
    # 🔥 AQUÍ REEMPLAZA CON TUS DATOS REALES
    # =========================================================================

    print("\n" + "📦" * 30)
    print("DATOS DE EJEMPLO - REEMPLAZA CON TUS DATOS")
    print("📦" * 30 + "\n")

    # Ejemplo: Si tienes tus datos en CSV
    """
    import pandas as pd
    
    # Cargar datos
    df_train = pd.read_csv('train.csv')
    df_test = pd.read_csv('test.csv')
    
    X_train_crudo = df_train['columna_texto'].tolist()
    y_train = df_train['columna_etiqueta'].tolist()
    X_test_crudo = df_test['columna_texto'].tolist()
    y_test = df_test['columna_etiqueta'].tolist()
    """

    # Datos de ejemplo (REEMPLAZA ESTO)
    X_train_crudo = [
        "Este producto es excelente me encantó",
        "Muy mal servicio no lo recomiendo",
        "Buena calidad precio correcto",
        "Terrible experiencia pésimo",
        "Genial lo volvería a comprar",
        "No vale la pena muy caro",
        "Excelente producto muy satisfecho",
        "Horrible no funciona bien",
        "Cumple con lo esperado bien",
        "Decepcionante no lo compren",
        "Perfecto justo lo que necesitaba",
        "Malísimo perdí mi dinero",
        "Recomendable buena compra",
        "Pésima calidad no sirve",
        "Increíble superó expectativas",
        "No recomiendo mala experiencia",
        "Muy bueno estoy contento",
        "Fatal no funciona nada",
        "Correcto hace su función",
        "Horrible muy decepcionado",
    ]

    y_train = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

    X_test_crudo = [
        "Producto excelente muy recomendable",
        "No lo compren muy malo",
        "Buena opción cumple expectativas",
        "Pésimo no sirve para nada",
        "Genial estoy muy contento",
    ]

    y_test = [1, 0, 1, 0, 1]

    print(f"X_train: {len(X_train_crudo)} muestras")
    print(f"X_test:  {len(X_test_crudo)} muestras")
    print(f"Clases:  {np.unique(y_train)}\n")

    # =========================================================================
    # EJECUTAR PIPELINE COMPLETO
    # =========================================================================

    modelo, representador, resultados = pipeline_completo(
        X_train_crudo,
        y_train,
        X_test_crudo,
        y_test,
        max_features=100,  # Aumenta con más datos
        min_df=1,  # Cambia a 2 con más datos
    )

    # =========================================================================
    # EJEMPLO DE PREDICCIÓN CON TEXTO NUEVO
    # =========================================================================

    print("\n" + "=" * 60)
    print("🔮 PREDICCIÓN CON TEXTOS NUEVOS")
    print("=" * 60)

    textos_nuevos = [
        "Este producto es increíble lo recomiendo",
        "Horrible experiencia no funciona",
        "Producto correcto cumple función",
    ]

    for i, texto in enumerate(textos_nuevos, 1):
        pred, conf = predecir_texto_nuevo(texto, modelo, representador)
        sentimiento = "POSITIVO ✅" if pred == 1 else "NEGATIVO ❌"

        print(f"\n{i}. Texto: {texto}")
        print(f"   Predicción: {sentimiento}")
        if conf:
            print(f"   Confianza: {conf:.2%}")

    # =========================================================================
    # INSTRUCCIONES FINALES
    # =========================================================================

    print("\n" + "=" * 60)
    print("✅ ¡COMPLETADO!")
    print("=" * 60)
    print("""
    Para usar con TUS datos:
    
    1. Reemplaza las líneas 324-347 con tus datos reales:
    
       import pandas as pd
       
       df = pd.read_csv('tus_datos.csv')
       X_train_crudo = df['columna_texto'].tolist()
       y_train = df['columna_etiquetas'].tolist()
       # etc...
    
    2. Ejecuta el script:
       
       python script_completo_todo_en_uno.py
    
    3. Usa los archivos generados:
       
       • modelo_final.pkl
       • representador_final.pkl
    
    ¡Listo! 🎉
    """)



📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦
DATOS DE EJEMPLO - REEMPLAZA CON TUS DATOS
📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦📦

X_train: 20 muestras
X_test:  5 muestras
Clases:  [0 1]


🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
PIPELINE COMPLETO - CLASIFICACIÓN DE TEXTO
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

PASO 1: LIMPIEZA
Limpiando X_train...
✓ 20 textos limpios
Limpiando X_test...
✓ 5 textos limpios

Ejemplo de limpieza:
  Original: Este producto es excelente me encantó...
  Limpio:   este producto es excelente me encantó

PASO 2: TOKENIZACIÓN + LEMATIZACIÓN + TF-IDF
Procesando X_train...
✓ Matriz TF-IDF: (20, 61)
✓ Vocabulario: 61 palabras

Procesando X_test con el MISMO representador...
✓ Matriz TF-IDF: (5, 61)

PASO 3: ENTRENAMIENTO
🤖 ENTRENANDO MODELO
Tipo: logistic
Muestras: 20
✅ Modelo entrenado

PASO 4: EVALUACIÓN CON X_TEST
🔍 EVALUANDO MODELO

✅ ACCURACY: 1.0000 (100.00%)

📊 MATRIZ DE CONFUSIÓN:
[[2 0]
 [0 3]]

📈 REPORTE DETALLADO:
              precision    recall  f1-score   support

           0    

In [29]:
X_test_procesado = representador.transform(X_test_crudo)


y_pred = modelo.predict(X_test_procesado)


print("Tamaño y_test:", len(y_test))
print("Tamaño y_pred:", len(y_pred))


Tamaño y_test: 5
Tamaño y_pred: 5


In [30]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average="weighted"))
print("Recall:", recall_score(y_test, y_pred, average="weighted"))
print("F1 Score:", f1_score(y_test, y_pred, average="weighted"))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print(
    "Reporte de clasificación:\n",
    classification_report(y_test, y_pred, zero_division=0),
)


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
ROC AUC Score: 1.0
Reporte de clasificación:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         3

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



**Guardar el modelo**

In [31]:
import joblib

## Librerías Usadas

In [32]:
import sys
from importlib import metadata

mods = {m.split(".")[0] for m in sys.modules.keys() if m and not m.startswith("_")}
stdlib = getattr(sys, "stdlib_module_names", set())
third_party = sorted(m for m in mods if m and (m not in stdlib))

pkg_map = {}
try:
    pkg_map = metadata.packages_distributions()  # top_mod -> [dist]
except Exception:
    pass

dists = sorted({(pkg_map.get(m) or [None])[0] or m for m in third_party})
print("Paquetes/Distribuciones detectadas:\n", "\n ".join(dists))


Paquetes/Distribuciones detectadas:
 Pygments
 asttokens
 charset-normalizer
 comm
 cramjam
 cython_runtime
 debugpy
 decorator
 executing
 fastparquet
 fsspec
 ipykernel
 ipython
 ipywidgets
 jedi
 joblib
 jupyter_client
 jupyter_core
 numpy
 packaging
 pandas
 parso
 platformdirs
 prompt_toolkit
 psutil
 pure_eval
 pydev_ipython
 pydevconsole
 pydevd
 pydevd_file_utils
 pydevd_plugins
 pydevd_tracing
 python-dateutil
 pytz
 pyzmq
 scikit-learn
 scipy
 six
 stack-data
 threadpoolctl
 tornado
 traitlets
 vscode
 wcwidth


---

## Referencias
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html)
- Ejemplo de Entrenamiento y selección de Modelo de machine learning entre varios modelos <https://joserzapata.github.io/post/ciencia-datos-proyecto-python/6-model_selection/>
- https://joserzapata.github.io/courses/python-ciencia-datos/python/
- https://joserzapata.github.io/courses/python-ciencia-datos/pandas/
- https://joserzapata.github.io/courses/python-ciencia-datos/machine-learning/

Docente: [Jose R. Zapata](https://joserzapata.github.io)

- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)